pyESM WORKFLOW

In [1]:
# create an instance of Interface class
# based on the model settings, this generates the model's input files to be filled
# and the blank SQL database

from src.frontend.interface import Interface

test = Interface(
    log_level='debug',
    log_format='minimal',
)

INFO | Interface | 'Interface' object initialization...
INFO | Interface.file_manager | 'FileManager' object generated.
INFO | Interface | 'Interface' object: loading settings.
DEBUG | Interface.file_manager | File 'settings.yml' loaded.
INFO | Interface | 'Interface' object: loading paths from settings.
INFO | Interface.model | 'Model' object initialization...
INFO | Interface.model | Warm start option enabled. Skipping model directory generation.
INFO | Interface.model.sql_manager | 'SQLManager' object initialization...
INFO | Interface.model.sql_manager | 'SQLManager' object initialized.
INFO | Interface.model.index | 'Index' object initialization...
DEBUG | Interface.model.index | Loading variables table headers and label headers to Index.
INFO | Interface.model.index | 'Index' object initialized.
INFO | Interface.model.database | 'Database' object initialization...
INFO | Interface.model.database | 'Database' object initialized.
INFO | Interface.model.problem | 'Problem' object in

In [2]:
# load model sets from the xlsx files filled by the user
# (data validation in excel according to the constant file is envisaged)
test.load_sets()

INFO | Interface.model.index | 'Index' object: loading new Sets to Index.
DEBUG | Interface.file_manager | Excel file 'sets.xlsx' loaded.
INFO | Interface.model | Warm start option enabled. Skipping loading sets to the SQL database.


In [ ]:
# once sets has compiled in excel and loaded in the database, blank database 
# is generated
test.generate_blank_sql_database()

In [ ]:
# generate input files to filled by the user
test.generate_blank_data_input_files()

In [ ]:
# load input file/s data into the sql database
test.load_data_files_to_database()

In [ ]:
test.model.index.sets.technologies

In [ ]:
test.model.index.variables.v

In [ ]:
test.model.database.sqltools.open_connection()

var = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        't_Name': ['Gas power plant'],
        'f_Name': ['Steel']
    }
)

filter_1 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_FLOWS',
    child_column_name='f_Name',
    parent_table_fields={
        'f_Category': ['Product flow'],
    }
)

filter_2 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_DATETIME',
    child_column_name='dt_Name',
    parent_table_fields={
        'dt_Name': [2023],
    }
)

var_custom_filter = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        **filter_1,
        **filter_2,
        **{'f_Name': ['Steel']},
    }
)

test.model.database.sqltools.close_connection()

# filter_1, filter_2
var_custom_filter

In [ ]:
# delete all the information included in the model data folder, if necessary
test.model.model_dir_cleanup()